## Deliverable 2. Create a Customer Travel Destinations Map.

In [6]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [18]:
# File to Load 
weather_data_to_load = "/users/barrybingham/Desktop/Class/World_Weather_Analysis/Vacation_Search/WeatherPy_Database.csv"
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(weather_data_to_load)
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Castro,BR,-24.7911,-50.0119,68.70,82,69,8.63,broken clouds
1,1,Puerto Ayora,EC,-0.7393,-90.3518,80.56,80,56,10.47,broken clouds
2,2,Nanakuli,US,21.3906,-158.1547,73.62,77,40,5.75,scattered clouds
3,3,Atuona,PF,-9.8000,-139.0333,78.78,77,36,15.64,light rain
4,4,Albany,US,42.6001,-73.9662,77.95,51,98,1.01,overcast clouds


In [19]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [20]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Puerto Ayora,EC,-0.7393,-90.3518,80.56,80,56,10.47,broken clouds
3,3,Atuona,PF,-9.8000,-139.0333,78.78,77,36,15.64,light rain
4,4,Albany,US,42.6001,-73.9662,77.95,51,98,1.01,overcast clouds
5,5,Progreso,MX,21.2833,-89.6667,88.11,52,77,9.73,broken clouds
7,7,Bathsheba,BB,13.2167,-59.5167,86.61,62,75,21.85,broken clouds
9,9,Cocobeach,GA,1.0002,9.5823,79.36,81,98,4.65,overcast clouds
15,15,Rikitea,PF,-23.1203,-134.9692,77.67,71,6,19.30,light rain
17,17,Bambous Virieux,MU,-20.3428,57.7575,80.22,73,75,17.27,light rain
23,23,Butaritari,KI,3.0707,172.7902,80.67,78,86,11.95,light rain
35,35,Carnarvon,AU,-24.8667,113.6333,75.27,73,0,12.66,clear sky


In [21]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                254
City                   254
Country                249
Lat                    254
Lng                    254
Max Temp               254
Humidity               254
Cloudiness             254
Wind Speed             254
Current Description    254
dtype: int64

In [25]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
best_cities_df = preferred_cities_df.dropna()
best_cities_df.count()

City_ID                249
City                   249
Country                249
Lat                    249
Lng                    249
Max Temp               249
Humidity               249
Cloudiness             249
Wind Speed             249
Current Description    249
dtype: int64

In [27]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = best_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Puerto Ayora,EC,80.56,broken clouds,-0.7393,-90.3518,
3,Atuona,PF,78.78,light rain,-9.8000,-139.0333,
4,Albany,US,77.95,overcast clouds,42.6001,-73.9662,
5,Progreso,MX,88.11,broken clouds,21.2833,-89.6667,
7,Bathsheba,BB,86.61,broken clouds,13.2167,-59.5167,
9,Cocobeach,GA,79.36,overcast clouds,1.0002,9.5823,
15,Rikitea,PF,77.67,light rain,-23.1203,-134.9692,
17,Bambous Virieux,MU,80.22,light rain,-20.3428,57.7575,
23,Butaritari,KI,80.67,light rain,3.0707,172.7902,
35,Carnarvon,AU,75.27,clear sky,-24.8667,113.6333,


In [49]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    hotels
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df       
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Puerto Ayora,EC,80.56,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
3,Atuona,PF,78.78,light rain,-9.8000,-139.0333,Villa Enata
4,Albany,US,77.95,overcast clouds,42.6001,-73.9662,Macho lodge
5,Progreso,MX,88.11,broken clouds,21.2833,-89.6667,Playa Linda Hotel
7,Bathsheba,BB,86.61,broken clouds,13.2167,-59.5167,Atlantis Hotel
...,...,...,...,...,...,...,...
701,Dogbo,BJ,90.57,overcast clouds,6.8167,1.7833,Maison des visiteurs / auberge chez ESI ONG/ p...
702,Vao,NC,76.91,overcast clouds,-22.6667,167.4833,Hotel Kou-Bugny
705,Codajas,BR,85.33,moderate rain,-3.8367,-62.0569,Atlântic Hotel
708,Kirakira,SB,79.63,scattered clouds,-10.4544,161.9205,SanBiz Lodge


In [52]:
# 7. Drop the rows where there is no Hotel Name.
best_hotels_df = hotel_df.dropna(subset=['Hotel Name'])
best_hotels_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Puerto Ayora,EC,80.56,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
3,Atuona,PF,78.78,light rain,-9.8000,-139.0333,Villa Enata
4,Albany,US,77.95,overcast clouds,42.6001,-73.9662,Macho lodge
5,Progreso,MX,88.11,broken clouds,21.2833,-89.6667,Playa Linda Hotel
7,Bathsheba,BB,86.61,broken clouds,13.2167,-59.5167,Atlantis Hotel
...,...,...,...,...,...,...,...
701,Dogbo,BJ,90.57,overcast clouds,6.8167,1.7833,Maison des visiteurs / auberge chez ESI ONG/ p...
702,Vao,NC,76.91,overcast clouds,-22.6667,167.4833,Hotel Kou-Bugny
705,Codajas,BR,85.33,moderate rain,-3.8367,-62.0569,Atlântic Hotel
708,Kirakira,SB,79.63,scattered clouds,-10.4544,161.9205,SanBiz Lodge


In [54]:
# 8a. Create the output File (CSV)
output_data_file = "/users/barrybingham/Desktop/Class/World_Weather_Analysis/Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
best_hotels_df.to_csv(output_data_file, index_label="City_ID")

In [55]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in best_hotels_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = best_hotels_df[["Lat", "Lng"]]

In [56]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = best_hotels_df[["Lat", "Lng"]]
max_temp = best_hotels_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))